# Quantum Support Vector Classifier (QSVC) Splitting

This notebook will explore the Qiskit implementation of a QSVC, and initial attempts to split the QSVC into parts to explain.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

### Imports

In [ ]:
# imports
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import clear_output

import os
import time

# import data class
from utilities.dataset_utils import DiabetesData

# import metrics for evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# qiskit imports
# circuit transpiler
from qiskit import transpile

# algorithm
from qiskit_machine_learning.algorithms import QSVC

# feature map
from qiskit.circuit.library import ZZFeatureMap

# fidelity
from qiskit_algorithms.state_fidelities import ComputeUncompute

# kernel
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# simulator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

### QSVC Overview
A Quantum Support Vector Classifier







is a hybrid quantum-classical machine learning algorithm used for classification. It encodes the classical data using a feature map, which maps the data onto qubits to take advantage of quantum phenomena. The encoded data is processed through a variational quantum circuit (consists of parameterized gates). These parameters are iteratively optimised using classical algorithms to minimise a cost function (difference between predicted and true labels). After processing, quantum circuit is measured, and results are post processed to produce predictions. 
This hybrid approach allows the VQC allows the combination of the power of quantum mechanics with classical optimisation techniques, offering advantages for complex classification problems that would be challenging for a purely classical approach

In [ ]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [ ]:
# setup backend simulator
backend = AerSimulator(method='statevector')

In [ ]:
# feature map
feature_map = ZZFeatureMap(feature_dimension=dataset.get_num_features(), reps=1, entanglement='circular')
feature_map = transpile(feature_map, backend=backend) # transpile to backend

In [ ]:
# sampler
sampler = Sampler.from_backend(backend)

In [ ]:
# fidelity
fidelity = ComputeUncompute(sampler=sampler)

In [ ]:
# kernel
kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [ ]:
# create the QSVC instance
qsvc = QSVC(
    quantum_kernel=kernel,
    backend=backend
)

### Model Fit on Simulator

In [ ]:
# get training and testing data
X_train, X_test, y_train, y_test = dataset.preprocess_data()

In [ ]:
# time how long it takes to train
start = time.time()

# fit the model
qsvc = qsvc.fit(X_train, y_train)

end = time.time()
elapsed = end - start

# print training time
print(f"Training Time: {round(elapsed)} secs")

### QSVC Evaluation

In [ ]:
train_score = qsvc.score(X_train, y_train)
test_score = qsvc.score(X_test, y_test)
train_score, test_score

In [ ]:
# predict
y_pred = qsvc.predict(X_test)

In [ ]:
# confusion matrix
def conf_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    heatmap = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
    plt.title('Confusion Matrix: QSVC')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [ ]:
# accuracy score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# classification report
report = classification_report(y_test, y_pred, zero_division=0, output_dict=False)

In [ ]:
# print metrics
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')
conf_matrix(y_test, y_pred)